# Daily Aggregations using Xarray
*(author: Grant Buster)*

In this notebook, we demonstrate how you can use ``xarray`` to quickly and easily compute daily statistics for a year of sup3rCC data. (If it seems strange to you that we're aggregating a dataset intended to improve resolution - don't worry, we think it's a bit strange too).

Requirements:

- Install rex: `pip install NREL-rex --upgrade`
- Install dask-distributed: `pip install distributed --upgrade`
- Initialize a dask client for parallel processing (see below)
- Set dask compute chunks appropriately (see below)

### Helpful tips

- Performance is really sensitive (perhaps unsurprisingly) to the dask compute chunk size you specify. The h5 chunks on disk are way too small and result in too many operations, adding too much overhead to performance. You can get worse performance with xarray + dask vs. a serial process when working if your compute chunks are too small. Here, we find that `(8760, 50000)` is a good compute chunk size. Note that the storage chunk shape on disk is `(2000, 500)`.

- The `memory_limit` argument is the limit *per worker*. If you are memory constrained, try using less workers and setting a lower memory limit and reducing the compute chunk size. Here, we're processing on a large NREL HPC node with 104 cores and 256 GB of memory.

- Setting up the full aggregate dataset lazily and then doing one `.compute()` call tended to break things. Smaller multiple compute calls seem to work better. 

In [1]:
import glob
import xarray as xr
from rex import Resource
import numpy as np
import pandas as pd
from dask.distributed import Client

In [2]:
client = Client(n_workers=10, memory_limit='20GB', threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 10,Total memory: 186.26 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35041,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 186.26 GiB
Comm: tcp://127.0.0.1:33475,Total threads: 1
Dashboard: http://127.0.0.1:34177/status,Memory: 18.63 GiB
Nanny: tcp://127.0.0.1:46249,


In [3]:
%%time

year = 2015
scenario = 'ecearth3cc_ssp245_r1i1p1f1'

fp_base = '/datasets/sup3rcc/conus_{scenario}/v0.2.2_beta/sup3rcc_conus_{scenario}_{group}_{year}.h5'
fp_pr = fp_base.replace('v0.2.2_beta', 'v0.2.2_beta/daily')

kwargs = dict(engine="rex", chunks={'time': 8784, 'gid': 50000})
xds_trh = xr.open_mfdataset(fp_base.format(scenario=scenario, group='trh', year=year), **kwargs)
xds_wind = xr.open_mfdataset(fp_base.format(scenario=scenario, group='wind', year=year), **kwargs)
xds_pr = xr.open_mfdataset(fp_pr.format(scenario=scenario, group='pr', year=year), **kwargs)

CPU times: user 82.5 ms, sys: 66.8 ms, total: 149 ms
Wall time: 840 ms


In [4]:
%%time
da = xds_trh['temperature_2m'].groupby("time.date").max("time")
ds_out = da.compute().to_dataset()

CPU times: user 19.6 s, sys: 6.66 s, total: 26.3 s
Wall time: 1min 14s


In [5]:
%%time
da = xds_trh['relativehumidity_2m'].groupby("time.date").min("time")
ds_out['relativehumidity_2m'] = da.compute()

CPU times: user 18.7 s, sys: 5.33 s, total: 24.1 s
Wall time: 38.8 s


In [6]:
%%time
da = xds_wind['windspeed_10m'].groupby("time.date").mean("time")
ds_out['windspeed_10m'] = da.compute() * 3.6  # m/s to km/hr

CPU times: user 24 s, sys: 5.62 s, total: 29.6 s
Wall time: 58.6 s


In [7]:
%%time
xds_pr['time'] = pd.to_datetime(xds_pr['time'].values - pd.Timedelta('12hr'))
xds_pr = xds_pr.rename({'time': 'date'})
ds_out['pr'] = xds_pr['pr'].compute() * 86400  # kg/m2s to mm/day

CPU times: user 2.19 s, sys: 3.32 s, total: 5.51 s
Wall time: 9.24 s


In [8]:
%%time
# reshape NREL data format from (time, gid) to (time, lat, lon) and set attrs

ds_out = ds_out.set_index(gid=['latitude', 'longitude'])
ds_out = ds_out.unstack('gid')
ds_out = ds_out.rename({'latitude': 'lat', 'longitude': 'lon', 'date': 'time'})

ds_out['temperature_2m'].attrs['aggregation'] = 'Daily maximum'
ds_out['relativehumidity_2m'].attrs['aggregation'] = 'Daily minimum'
ds_out['windspeed_10m'].attrs['aggregation'] = 'Daily average'
ds_out['windspeed_10m'].attrs['units'] = 'km/hr'
ds_out['pr'].attrs['aggregation'] = 'Daily accumulation'
ds_out['pr'].attrs['units'] = 'mm/day'

ds_out['time'] = pd.to_datetime(ds_out['time'].values)
ds_out = ds_out.drop_vars('time_index', errors='ignore')

CPU times: user 22.2 s, sys: 3.62 s, total: 25.8 s
Wall time: 24.8 s


In [9]:
%%time

encoding = {"temperature_2m": {"dtype": "int16", "scale_factor": 0.01, '_FillValue': 100, 'chunksizes': (100, 100, 100)},
            "windspeed_10m": {"dtype": "int16", "scale_factor": 0.01, '_FillValue': 120, 'chunksizes': (100, 100, 100)},
            "relativehumidity_2m": {"dtype": "uint16", "scale_factor": 0.01, '_FillValue': 101, 'chunksizes': (100, 100, 100)},
            "pr": {"dtype": "float32", 'chunksizes': (100, 100, 100)},
           }

fp_out = f'/scratch/gbuster/sup3rcc_fwi/sup3rcc_test_daily_{scenario}_{year}.nc'
ds_out.to_netcdf(fp_out, format='NETCDF4', engine="h5netcdf", encoding=encoding)

CPU times: user 15.8 s, sys: 11.9 s, total: 27.6 s
Wall time: 26.8 s


In [10]:
ds_out

<xarray.Dataset> Size: 27GB
Dimensions:              (lat: 1000, lon: 2300, time: 365)
Coordinates:
  * lat                  (lat) float32 4kB 23.17 23.2 23.23 ... 51.19 51.21
  * lon                  (lon) float32 9kB -129.0 -129.0 ... -64.38 -64.35
    elevation            (lat, lon) int16 5MB 0 0 0 0 0 ... 727 645 541 705 600
    timezone             (lat, lon) float32 9MB -9.0 -9.0 -9.0 ... -5.0 -5.0
    country              (lat, lon) |S18 41MB b'nan' b'nan' ... b'Canada'
    state                (lat, lon) |S20 46MB b'nan' b'nan' ... b'Qu\xc3\xa9bec'
    county               (lat, lon) |S33 76MB b'nan' ... b'Minganie--Basse-C\...
    offshore             (lat, lon) int16 5MB 1 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0
    eez                  (lat, lon) int16 5MB 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1
  * time                 (time) datetime64[ns] 3kB 2015-01-01 ... 2015-12-31
Data variables:
    temperature_2m       (time, lat, lon) float64 7GB 19.94 20.02 ... -16.45
    relativehumidity_2m  (time, lat, lon) float64 7GB 65.34 65.06 ... 67.03
    windspeed_10m        (time, lat, lon) float64 7GB 19.11 19.02 ... 16.81
    pr                   (time, lat, lon) float64 7GB 0.1235 0.1163 ... 0.0 0.0